# Lab-04-1 Multivariate Linear regression

In [1]:
import numpy as np
import pandas as pd
import torch
from torch import optim

## Data

In [2]:
x_train = torch.FloatTensor([[73, 80, 75], 
                             [93, 88, 93], 
                             [89, 91, 90], 
                             [96, 98, 100], 
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

## Model without torch.nn

### Model Initialization

In [3]:
W = torch.randn(x_train.size(1), requires_grad=True)
b = torch.randn(1, requires_grad=True)
W, b

(tensor([ 0.6190, -0.4464, -0.5915], requires_grad=True),
 tensor([1.1979], requires_grad=True))

### Optimizer

In [4]:
optimizer = optim.SGD([W, b], lr=1e-5)

### Predict

In [5]:
x_train.matmul(W) + b

tensor([-33.6915, -35.5298, -37.5706, -42.2776, -24.4840],
       grad_fn=<AddBackward0>)

### Train

In [6]:
n_epochs = 20
for epoch in range(n_epochs + 1):
    
    output = (x_train.matmul(W) + b).unsqueeze(1)
    
    loss = torch.mean((output - y_train)**2)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    print('Epoch {:4d}/{} y_pred: {} Cost: {:.6f}'.format(
        epoch, n_epochs, output.squeeze().detach(), loss.item()
    ))
    print('W: {} b: {:.2f}'.format(W.detach().numpy(), b.item()))

Epoch    0/20 y_pred: tensor([-33.6915, -35.5298, -37.5706, -42.2776, -24.4840]) Cost: 42988.960938
W: [ 0.97282803 -0.09282294 -0.23350555] b: 1.20
Epoch    1/20 y_pred: tensor([47.2797, 61.7906, 58.3213, 62.1463, 49.7467]) Cost: 13476.015625
W: [ 1.1708956   0.10517381 -0.03307883] b: 1.20
Epoch    2/20 y_pred: tensor([ 92.6127, 116.2766, 112.0077, 120.6094,  91.3056]) Cost: 4225.273438
W: [1.2817597  0.216054   0.07913034] b: 1.21
Epoch    3/20 y_pred: tensor([117.9932, 146.7811, 142.0649, 153.3409, 114.5727]) Cost: 1325.655518
W: [1.343802   0.2781608  0.14194974] b: 1.21
Epoch    4/20 y_pred: tensor([132.2029, 163.8594, 158.8928, 171.6660, 127.5990]) Cost: 416.778900
W: [1.3785105  0.31296116 0.17711765] b: 1.21
Epoch    5/20 y_pred: tensor([140.1587, 173.4207, 168.3142, 181.9257, 134.8917]) Cost: 131.893616
W: [1.397916   0.33247364 0.19680446] b: 1.21
Epoch    6/20 y_pred: tensor([144.6130, 178.7736, 173.5890, 187.6697, 138.9744]) Cost: 42.596901
W: [1.4087538  0.343427   0.2078

## Manual Model

In [7]:
import torch
from torch import optim
import torch.nn as nn

In [8]:
class ManualLinearRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.W = nn.Parameter(torch.randn(x_train.size(1), requires_grad=True))
        self.b = nn.Parameter(torch.randn(1, requires_grad=True))
    
    def forward(self, x):
        return (x.matmul(self.W) + self.b).unsqueeze(1)

In [9]:
model = ManualLinearRegressor()
optimizer = optim.SGD(model.parameters(), lr=1e-5)
criterion = nn.MSELoss()

In [10]:
n_epochs = 20
for epoch in range(n_epochs):
    model.train()

    output = model(x_train)

    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    print('Epoch {:4d}/{} y_pred: {} Cost: {:.6f}'.format(
        epoch, n_epochs, output.squeeze().detach(), loss.item()
    ))

Epoch    0/20 y_pred: tensor([117.8605, 158.7107, 147.2782, 163.2412, 123.1134]) Cost: 871.438293
Epoch    1/20 y_pred: tensor([129.2317, 172.3746, 160.7433, 177.9041, 133.5349]) Cost: 289.546570
Epoch    2/20 y_pred: tensor([135.5989, 180.0240, 168.2823, 186.1135, 139.3688]) Cost: 107.145340
Epoch    3/20 y_pred: tensor([139.1645, 184.3061, 172.5033, 190.7098, 142.6343]) Cost: 49.964241
Epoch    4/20 y_pred: tensor([141.1615, 186.7028, 174.8667, 193.2832, 144.4618]) Cost: 32.032772
Epoch    5/20 y_pred: tensor([142.2804, 188.0442, 176.1901, 194.7242, 145.4842]) Cost: 26.403889
Epoch    6/20 y_pred: tensor([142.9077, 188.7946, 176.9314, 195.5311, 146.0560]) Cost: 24.631245
Epoch    7/20 y_pred: tensor([143.2597, 189.2141, 177.3466, 195.9830, 146.3753]) Cost: 24.067318
Epoch    8/20 y_pred: tensor([143.4575, 189.4485, 177.5793, 196.2361, 146.5534]) Cost: 23.882330
Epoch    9/20 y_pred: tensor([143.5692, 189.5791, 177.7099, 196.3781, 146.6524]) Cost: 23.816011
Epoch   10/20 y_pred: tenso

## Nested Models

In [11]:
import torch
from torch import optim
import torch.nn as nn

In [12]:
class LayerLinearRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(x_train.size(1), 1)
    
    def forward(self, x):
        return self.linear(x)

In [13]:
model = LayerLinearRegressor()
optimizer = optim.SGD(model.parameters(), lr=1e-5)
criterion = nn.MSELoss()

In [14]:
n_epochs = 20
for epoch in range(n_epochs):
    model.train()

    output = model(x_train)

    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    print('Epoch {:4d}/{} y_pred: {} Cost: {:.6f}'.format(
        epoch, n_epochs, output.squeeze().detach(), loss.item()
    ))

Epoch    0/20 y_pred: tensor([35.9298, 38.7093, 40.5426, 42.4328, 29.9311]) Cost: 18092.789062
Epoch    1/20 y_pred: tensor([ 88.4508, 101.8374, 102.7428, 110.1674,  78.0823]) Cost: 5675.182129
Epoch    2/20 y_pred: tensor([117.8549, 137.1807, 137.5663, 148.0894, 105.0407]) Cost: 1782.919678
Epoch    3/20 y_pred: tensor([134.3168, 156.9684, 157.0626, 169.3205, 120.1341]) Cost: 562.900757
Epoch    4/20 y_pred: tensor([143.5328, 168.0471, 167.9777, 181.2069, 128.5847]) Cost: 180.486725
Epoch    5/20 y_pred: tensor([148.6921, 174.2500, 174.0886, 187.8616, 133.3162]) Cost: 60.618214
Epoch    6/20 y_pred: tensor([151.5802, 177.7230, 177.5097, 191.5872, 135.9656]) Cost: 23.043730
Epoch    7/20 y_pred: tensor([153.1967, 179.6677, 179.4249, 193.6730, 137.4493]) Cost: 11.264114
Epoch    8/20 y_pred: tensor([154.1014, 180.7567, 180.4970, 194.8406, 138.2803]) Cost: 7.569697
Epoch    9/20 y_pred: tensor([154.6074, 181.3667, 181.0972, 195.4943, 138.7459]) Cost: 6.409578
Epoch   10/20 y_pred: tensor

### use torch.nn.functional as F

In [15]:
import torch.nn.functional as F

In [16]:
class LayerLinearRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(x_train.size(1), 1)
    
    def forward(self, x):
        return self.linear(x)

In [17]:
model = LayerLinearRegressor()
optimizer = optim.SGD(model.parameters(), lr=1e-5)

In [18]:
n_epochs = 20
for epoch in range(n_epochs):
    model.train()

    output = model(x_train)

    loss = F.mse_loss(output, y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    print('Epoch {:4d}/{} y_pred: {} Cost: {:.6f}'.format(
        epoch, n_epochs, output.squeeze().detach(), loss.item()
    ))

Epoch    0/20 y_pred: tensor([15.7411, 18.3557, 18.4475, 19.1016, 14.6982]) Cost: 23986.962891
Epoch    1/20 y_pred: tensor([76.2238, 91.0521, 90.0762, 97.1034, 70.1474]) Cost: 7519.571289
Epoch    2/20 y_pred: tensor([110.0856, 131.7523, 130.1784, 140.7737, 101.1916]) Cost: 2357.917480
Epoch    3/20 y_pred: tensor([129.0435, 154.5390, 152.6301, 165.2231, 118.5722]) Cost: 740.012878
Epoch    4/20 y_pred: tensor([139.6571, 167.2966, 165.1999, 178.9114, 128.3032]) Cost: 232.885101
Epoch    5/20 y_pred: tensor([145.5990, 174.4392, 172.2372, 186.5748, 133.7514]) Cost: 73.927711
Epoch    6/20 y_pred: tensor([148.9256, 178.4382, 176.1771, 190.8654, 136.8018]) Cost: 24.101908
Epoch    7/20 y_pred: tensor([150.7877, 180.6772, 178.3828, 193.2674, 138.5098]) Cost: 8.484011
Epoch    8/20 y_pred: tensor([151.8301, 181.9309, 179.6177, 194.6122, 139.4661]) Cost: 3.588177
Epoch    9/20 y_pred: tensor([152.4135, 182.6329, 180.3090, 195.3650, 140.0018]) Cost: 2.053119
Epoch   10/20 y_pred: tensor([152.

## Sequential Model

In [19]:
import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F

In [20]:
model = nn.Sequential(nn.Linear(x_train.size(1), 1))
optimizer = optim.SGD(model.parameters(), lr=1e-5)

In [21]:
n_epochs = 20
for epoch in range(n_epochs):
    model.train()

    output = model(x_train)

    loss = F.mse_loss(output, y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    print('Epoch {:4d}/{} y_pred: {} Cost: {:.6f}'.format(
        epoch, n_epochs, output.squeeze().detach(), loss.item()
    ))

Epoch    0/20 y_pred: tensor([51.4497, 58.2111, 59.2689, 64.6877, 43.2571]) Cost: 13550.974609
Epoch    1/20 y_pred: tensor([ 96.8978, 112.8382, 113.0929, 123.3008,  84.9242]) Cost: 4252.589844
Epoch    2/20 y_pred: tensor([122.3421, 143.4222, 143.2269, 156.1160, 108.2526]) Cost: 1338.037354
Epoch    3/20 y_pred: tensor([136.5870, 160.5454, 160.0977, 174.4879, 121.3136]) Cost: 424.476959
Epoch    4/20 y_pred: tensor([144.5617, 170.1324, 169.5429, 184.7736, 128.6265]) Cost: 138.121902
Epoch    5/20 y_pred: tensor([149.0260, 175.5000, 174.8307, 190.5320, 132.7211]) Cost: 48.362446
Epoch    6/20 y_pred: tensor([151.5250, 178.5055, 177.7911, 193.7559, 135.0139]) Cost: 20.225183
Epoch    7/20 y_pred: tensor([152.9236, 180.1885, 179.4483, 195.5607, 136.2979]) Cost: 11.402960
Epoch    8/20 y_pred: tensor([153.7063, 181.1310, 180.3761, 196.5710, 137.0173]) Cost: 8.635107
Epoch    9/20 y_pred: tensor([154.1440, 181.6590, 180.8953, 197.1366, 137.4204]) Cost: 7.764999
Epoch   10/20 y_pred: tensor

# Lab-04-2 Loading Data

## Mini-Batch

### Dataset

In [22]:
from torch.utils.data import Dataset

In [23]:
x_train = torch.FloatTensor([[73, 80, 75], 
                             [93, 88, 93], 
                             [89, 91, 90], 
                             [96, 98, 100], 
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [24]:
class CustomDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x = x_data
        self.y = y_data
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return (self.x[idx], self.y[idx])

In [25]:
train_data = CustomDataset(x_train, y_train)

In [26]:
train_data[0]

(tensor([73., 80., 75.]), tensor([152.]))

### DataLoader

In [27]:
from torch.utils.data import DataLoader

In [28]:
train_loader = DataLoader(dataset=train_data, batch_size=2, shuffle=True) 

In [29]:
train_loader

### Mini-Batch Gradient Descent

In [30]:
import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [31]:
for batch_idx, samples in enumerate(train_loader):
    print(batch_idx)
    print(samples)

0
[tensor([[73., 80., 75.],
        [73., 66., 70.]]), tensor([[152.],
        [142.]])]
1
[tensor([[93., 88., 93.],
        [89., 91., 90.]]), tensor([[185.],
        [180.]])]
2
[tensor([[ 96.,  98., 100.]]), tensor([[196.]])]


In [32]:
n_epochs = 20
for epoch in range(n_epochs):
    for batch_idx, samples in enumerate(train_loader):
        x_train, y_train = samples
        
        output = model(x_train)
        
        loss = F.mse_loss(output, y_train)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        print('Epoch {:4d}/{} Batch: {}/{} Cost: {:.6f}'.format(
            epoch, n_epochs, batch_idx+1, len(train_loader), loss.item()
        ))

Epoch    0/20 Batch: 1/3 Cost: 4.751957
Epoch    0/20 Batch: 2/3 Cost: 6.606174
Epoch    0/20 Batch: 3/3 Cost: 22.535255
Epoch    1/20 Batch: 1/3 Cost: 9.914578
Epoch    1/20 Batch: 2/3 Cost: 5.700682
Epoch    1/20 Batch: 3/3 Cost: 16.453857
Epoch    2/20 Batch: 1/3 Cost: 8.844458
Epoch    2/20 Batch: 2/3 Cost: 6.263742
Epoch    2/20 Batch: 3/3 Cost: 11.116164
Epoch    3/20 Batch: 1/3 Cost: 5.768337
Epoch    3/20 Batch: 2/3 Cost: 11.798955
Epoch    3/20 Batch: 3/3 Cost: 4.255417
Epoch    4/20 Batch: 1/3 Cost: 11.610088
Epoch    4/20 Batch: 2/3 Cost: 7.249529
Epoch    4/20 Batch: 3/3 Cost: 2.300960
Epoch    5/20 Batch: 1/3 Cost: 16.638527
Epoch    5/20 Batch: 2/3 Cost: 9.157446
Epoch    5/20 Batch: 3/3 Cost: 2.090781
Epoch    6/20 Batch: 1/3 Cost: 8.691968
Epoch    6/20 Batch: 2/3 Cost: 1.279174
Epoch    6/20 Batch: 3/3 Cost: 24.079973
Epoch    7/20 Batch: 1/3 Cost: 7.868084
Epoch    7/20 Batch: 2/3 Cost: 14.699789
Epoch    7/20 Batch: 3/3 Cost: 4.070806
Epoch    8/20 Batch: 1/3 Cost: 1